In [1]:
using PyCall

# スクレイピングしていい？

参考　

- [Pythonでスクレイピングのルール(robots.txt)をチェックするツールを作る](https://qiita.com/Broccolingual/items/aa1f48454b9972b82d63)

In [2]:
rob = pyimport("urllib.robotparser")

rp = rob.RobotFileParser()
rp.set_url("https://www.google.com/finance/robots.txt")
rp.read()

# robots.txtの情報から調査したいURL、User-Agentでクロール可能かを調べる
user_agent = "*"
url = "https://www.google.com/finance/quote/*"
result = rp.can_fetch(user_agent, url)
print(result)

true

# web画面スクレイピング

- [黒木さん gist](https://nbviewer.org/gist/genkuroki/c26f22d3a06a69f917fc98bb07c5c90c)

In [3]:
bs4 = pyimport("bs4")
requests = pyimport("requests")

PyObject <module 'requests' from 'C:\\Users\\salzl\\.julia\\conda\\3\\lib\\site-packages\\requests\\__init__.py'>

In [4]:
url_dict = Dict(
    "nikkei" => [
        "https://www.google.com/finance/quote/NI225:INDEXNIKKEI?hl=ja"
    ],
    "voo" => [
         "https://www.google.com/finance/quote/VOO:NYSEARCA?hl=ja%2Frobots.txt"
    ],
    "usd_jpy" => [
        "https://www.google.com/finance/quote/USD-JPY?hl=ja%2Frobots.txt"
    ]
)

Dict{String, Vector{String}} with 3 entries:
  "voo"     => ["https://www.google.com/finance/quote/VOO:NYSEARCA?hl=ja%2Frobo…
  "nikkei"  => ["https://www.google.com/finance/quote/NI225:INDEXNIKKEI?hl=ja"]
  "usd_jpy" => ["https://www.google.com/finance/quote/USD-JPY?hl=ja%2Frobots.tx…

In [5]:
response = requests.get(url_dict["voo"][1])
soup = bs4.BeautifulSoup(response.content)
soup.select(".kf1m0 div")[1].text

"\$402.92"

In [6]:
soup.select(".ygUjEc")[1].text

"Pre-market:\$402.60(0.079%)-0.32"

In [7]:
# メッセージ作成
msg = ""
for ( ticker, url ) in url_dict
    response = requests.get(url_dict[ticker][1])
    soup = bs4.BeautifulSoup(response.content)
    
    msg *= "$ticker : "
    # 株価
    msg *= soup.select(".kf1m0 div")[1].text * "\n "
    # 日付
    msg *= soup.select(".ygUjEc")[1].text * "\n"
end
print(msg)

voo : $402.92
 Pre-market:$402.60(0.079%)-0.32
nikkei : 26,843.49
 4月13日, 15:15:02 GMT+9 · INDEXNIKKEI · 免責条項
usd_jpy : 125.7885
 Apr 13, 1:27:25 PM UTC · Disclaimer


# slackに通知

- [Slack API を使用してメッセージを投稿する](https://zenn.dev/kou_pg_0131/articles/slack-api-post-message)

In [8]:
"""
curl -X POST 'https://slack.com/api/chat.postMessage' \
-d 'token=TOKEN' \
-d 'channel=#株価' \
-d 'text=*テキスト*'
"""

"curl -X POST 'https://slack.com/api/chat.postMessage' -d 'token=TOKEN' -d 'channel=#株価' -d 'text=*テキスト*'\n"

In [9]:
using HTTP

TOKEN = ""

""

In [10]:
params = Dict(
    "channel" => "#株価",
    "text" => "*テキスト*"
)

headers = Dict(
    "Authorization" => "Bearer $TOKEN",
)

# 書類一覧APIの呼び出し
res = HTTP.post(
    "https://slack.com/api/chat.postMessage",
    headers;
    require_ssl_verification = false,
    query = params,
)

# レスポンスの表示
println( String( res.body ) )

{"ok":false,"error":"not_authed"}


# GitHub Actions で使うで

- [Git Hub Actions入門](https://zenn.dev/hashito/articles/7c292f966c0b59)
- [GitHub Actions ワークフローで処理を定期実行する方法](https://gotohayato.com/content/514/)
- [ゴマちゃん　リポジトリ](https://github.com/terasakisatoshi/MyWorkflow.jl)

# julia 文法

In [11]:
# dict の使い方
robots_txt_url = Dict(
    "nikkei" => [
        "https://www.google.com/finance/quote/NI225:INDEXNIKKEI?hl=ja/robots.txt"
    ],
    "voo" => [
        "https://www.google.com/finance/quote/VOO:NYSEARCA?hl=ja%2Frobots.txt"
    ]
)

for (i,j) in robots_txt_url
    @show i,j
end

(i, j) = ("voo", ["https://www.google.com/finance/quote/VOO:NYSEARCA?hl=ja%2Frobots.txt"])
(i, j) = ("nikkei", ["https://www.google.com/finance/quote/NI225:INDEXNIKKEI?hl=ja/robots.txt"])
